In [54]:
import numpy as np
import pandas as pd


In [55]:
df = pd.read_csv('data/2022-01-05.csv', index_col=0)
df.head(5)


,price,bd_num,ba_num,sqft,address,zestimate,Building,Calendar,Heating,Snowflake,Parking,Lot,PriceSquareFeet,url,zipcode
0,"$279,900",3,1,"1,324","3072 Beechwood Blvd,Pittsburgh, PA 15217","$275,900",Single family residence,Built in 1940,"Natural gas, baseboard","Wall unit(s), window a/c",Open parking,"3,049 sqft",$211 price/sqft,https://www.zillow.com/homedetails/3072-Beechw...,15106
1,"$258,900",4,3,"2,854","304 Washington Ave,New Galilee, PA 16141",Contact Agent,Single family residence,Built in 1957,"Natural gas, forced air",Central air,2 Attached garage spaces,0.42 Acres,$91 price/sqft,https://www.zillow.com/homedetails/304-Washing...,15106
2,"$167,000",5,2,"3,698","1000 Ridge Ave,New Cumberland, WV 26047","$160,800",Single family residence,Built in 1875,"Forced air, fireplace(s), gas",None,3 Garage spaces,0.51 Acres,$45 price/sqft,https://www.zillow.com/homedetails/1000-Ridge-...,15106
3,"$350,000",3,3,"2,180","165 Jamison Ln,Monroeville, PA 15146","$350,100",Single family residence,Built in 1965,"Electric, radiant","Central air, electric",3 Attached garage spaces,0.25 Acres,$161 price/sqft,https://www.zillow.com/homedetails/165-Jamison...,15106
4,"$219,900",3,3,"2,994","35 Northgait Dr,Slippery Rock, PA 16057",None,Single family residence,Built in 1985,No data,No data,2 Attached garage spaces,1.57 Acres,$73 price/sqft,https://www.zillow.com/homedetails/35-Northgai...,15106


[Feature Engineering for Categorical Attributes](https://dotdata.com/blog/feature-engineering-for-categorical-attributes/)

[One-Hot Encoding a Feature on a Pandas Dataframe: Examples](https://queirozf.com/entries/one-hot-encoding-a-feature-on-a-pandas-dataframe-an-example)

### Category column process:

- Building : One-hot
- Heating : One-hot with combination
- Snowflake : One-hot with combination
- Parking: label encoding
- Zipcode: One-hot

### Insignificant features:

- address


## Column classify


In [56]:
# price, bd_num, ba_num, sqft, zestimate, Calendar, Lot, PriceSquareFeet
num_cols = ['price', 'bd_num', 'ba_num', 'sqft', 'Calendar']
one_hot_cols = ['Building', 'zipcode']
one_hot_unstruc_cols = ['Heating', 'Snowflake']
ord_unstruc_cols = ['Parking']
removed_cols = ['address', 'zestimate', 'PriceSquareFeet', 'url']


## Column data process function


In [57]:
def extract_num(num_str):
    '''Extract number value from string'''
    num_str = str(num_str)
    ans = ''
    for letter in num_str:
        if letter.isdecimal() or letter == '.':
            ans += letter
    return float(ans) if ans else None


def one_hot_transform(df, col_name):
    import pandas as pd
    # get one hot coding matrix
    one_hot_df = pd.get_dummies(df[col_name], prefix=col_name)
    # merge one hot with df
    df = pd.concat([df, one_hot_df], axis=1)
    # delete original feature
    df.drop([col_name], axis=1, inplace=True)
    return df


def one_hot_transform_unstructure(df, col_name, separator):
    # first loop through all rows to get all titles
    titles = set()
    for names in df[col_name]:
        for title in names.split(separator):
            titles.add(title.strip())
    titles = list(titles)
    # create new column based on existance of the title in col_name
    for title in titles:
        df[col_name + '_' +
            title] = df[col_name].apply(lambda x: 1 if title in x else 0)
    # delete original feature
    df.drop([col_name], axis=1, inplace=True)
    return df

# special for one col


def parking_score(info):
    '''
    scale = {
        'garage': 10,
        'covered': 8,
        'parking': 6
    }
    '''
    num = extract_num(info)
    num = num if num else 1
    # print(num)
    if 'garage' in info.lower():
        return 10 * num
    elif 'covered' in info.lower():
        return 8 * num
    elif 'parking' in info.lower():
        return 6 * num
    return 0


def lot_area(info):
    info = str(info)
    multiply = 43560 if 'acres' in info.lower() else 1
    val = extract_num(info)
    res = val*multiply if val else None
    return res


## Remove cols

In [58]:
for col in removed_cols:
    del df[col]

## Apply functions to columns


In [59]:
df['Parking'] = df['Parking'].apply(parking_score)
df['Lot'] = df['Lot'].apply(lot_area)
for col in num_cols:
    df[col] = df[col].apply(extract_num)
for col in one_hot_cols:
    df = one_hot_transform(df, col)
for col in one_hot_unstruc_cols:
    df = one_hot_transform_unstructure(df, col, ',')

## Replace None with average

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 91 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   price                                        783 non-null    float64
 1   bd_num                                       686 non-null    float64
 2   ba_num                                       684 non-null    float64
 3   sqft                                         545 non-null    float64
 4   Calendar                                     761 non-null    float64
 5   Parking                                      783 non-null    float64
 6   Lot                                          623 non-null    float64
 7   Building_Detached                            783 non-null    uint8  
 8   Building_Duplex                              783 non-null    uint8  
 9   Building_Multifamily                         783 non-null    uint8  
 10  Bu

In [61]:
for col in df.columns:
    df[col].fillna(df[col].mean(), inplace=True)
df


,price,bd_num,ba_num,sqft,Calendar,Parking,Lot,Building_Detached,Building_Duplex,Building_Multifamily,...,Snowflake_window a/c,Snowflake_electric,Snowflake_Central a/c,Snowflake_Other,Snowflake_Central air,Snowflake_wall unit(s),Snowflake_Attic fan,Snowflake_Electric,Snowflake_window unit(s),Snowflake_Central
0,279900.0,3.0,1.0,1324.0,1940.0,6.0,3049.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,258900.0,4.0,3.0,2854.0,1957.0,20.0,18295.2,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,167000.0,5.0,2.0,3698.0,1875.0,30.0,22215.6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,350000.0,3.0,3.0,2180.0,1965.0,30.0,10890.0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
4,219900.0,3.0,3.0,2994.0,1985.0,20.0,68389.2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,395000.0,3.0,3.0,1792.0,1936.0,20.0,15681.6,0,0,0,...,0,0,0,0,1,0,0,0,0,1
779,149900.0,3.0,2.0,1144.0,1930.0,6.0,3584.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
780,274900.0,4.0,3.0,2433.0,1904.0,20.0,6699.0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
781,269000.0,3.0,2.0,1328.0,1950.0,20.0,34848.0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 91 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   price                                        783 non-null    float64
 1   bd_num                                       783 non-null    float64
 2   ba_num                                       783 non-null    float64
 3   sqft                                         783 non-null    float64
 4   Calendar                                     783 non-null    float64
 5   Parking                                      783 non-null    float64
 6   Lot                                          783 non-null    float64
 7   Building_Detached                            783 non-null    uint8  
 8   Building_Duplex                              783 non-null    uint8  
 9   Building_Multifamily                         783 non-null    uint8  
 10  Bu

In [63]:
df.to_csv('test.csv')
